### Metagenomics

In [3]:
import os
import pandas as pd
import qiime2 as q2
import requests

from qiime2 import Visualization

data_dir = 'CE'
    
%matplotlib inline

In [2]:
def fetch_ipath(ids: list, img_output_path: str, verbose: bool = False):
    """Fetches a enriched pathways map from iPATH3 for given IDs."""
    url = 'https://pathways.embl.de/mapping.cgi'
    
    # remove colon from EC names
    if ':' in ids[0]:
        ids = [x.replace(':', '') for x in ids]
    
    if verbose:
        print(f'Fetching iPATH3 diagram for ids: {ids}')
    params = {
        'default_opacity': 0.6,
        'export_type': 'svg',
        'selection': '\n'.join(ids)
    }   
    response = requests.get(url=url, params=params)
    
    with open(img_output_path, 'wb') as img:
        img.write(response.content)

In [3]:
picrust_env = '/opt/conda/envs/picrust2/bin'

#### Functional inference

In [6]:
%%script env picrust_env="$picrust_env" data_dir="$data_dir" bash

# append the env location to PATH so that qiime
# can find all required executables
export PATH=$picrust_env:$PATH

$picrust_env/qiime picrust2 full-pipeline \
    --i-seq $data_dir/dada2_rep_set_filtered.qza \
    --i-table $data_dir/dada2_table_align_filtered.qza \
    --output-dir $data_dir/picrust2_results \
    --p-placement-tool sepp \
    --p-threads 2 \
    --p-hsp-method pic \
    --p-max-nsti 2 

Usage: qiime picrust2 full-pipeline [OPTIONS]

  QIIME 2 plugin for default 16S PICRUSt2 pipeline

Inputs:
  --i-table ARTIFACT FeatureTable[Frequency]
                       The feature table containing sequence abundances per
                       sample.                                      [required]
  --i-seq ARTIFACT FeatureData[Sequence]
                       Sequences (e.g. ASVs or representative OTUs)
                       corresponding to the abundance table given.  [required]
Parameters:
  --p-threads INTEGER  Number of threads/processes to use during workflow.
    Range(1, None)                                                [default: 1]
  --p-hsp-method TEXT Choices('mp', 'emp_prob', 'pic', 'scp',
    'subtree_average') Which hidden-state prediction method to use.
                                                               [default: 'mp']
  --p-placement-tool TEXT Choices('epa-ng', 'sepp')
                       Placement tool to use when placing sequences into
     

CalledProcessError: Command 'b'\n# append the env location to PATH so that qiime\n# can find all required executables\nexport PATH=$picrust_env:$PATH\n\n$picrust_env/qiime picrust2 full-pipeline \\\n    --i-seq $data_dir/dada2_rep_set_filtered.qza \\\n    --i-table $data_dir/dada2_table_align_filtered.qza \\\n    --output-dir $data_dir/picrust2_results \\\n    --p-placement-tool sepp \\\n    --p-threads 2 \\\n    --p-hsp-method pic \\\n    --p-max-nsti 2 \n'' returned non-zero exit status 1.

In [1]:
Test test..

SyntaxError: invalid syntax (557426884.py, line 1)

In [2]:
change 1,2,3 blibli blabla

SyntaxError: invalid syntax (3039516243.py, line 1)